In [1]:
import numpy as np
import xarray as xr
import sys 
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading US biological data ...


In [2]:
import pandas as pd 

In [3]:
%%time
out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
df = epro_2019.process_length_data(out, 0)
df.head()

CPU times: user 291 ms, sys: 3.76 ms, total: 295 ms
Wall time: 296 ms


,Sex,Length,Male_ind,Female_ind,n,meanlen,stdlen,TS_lin,TS_sd,TS_log,nM,nF
Haul,,,,,,,,,,,,
1,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 21....","[148, 149, 150, 151, 152, 153, 154, 155, 156, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",342,23.385965,2.125845,-40.585259,0.747452,-40.654102,194,148
3,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[18.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19....","[128, 129, 130, 131, 132, 133, 134, 135, 136, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",535,20.949533,1.400667,-41.557179,0.704258,-41.600977,97,128
7,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 21.0, 21....","[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 8...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",167,22.239521,1.247695,-41.043925,0.489027,-41.071140,95,72
8,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21....","[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",161,23.273292,1.781809,-40.637619,0.638929,-40.686783,93,68
9,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 21.0, 21....","[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",127,22.803150,3.023819,-40.764991,0.883306,-40.892414,64,63


In [4]:
sys.getsizeof(df)

246648

In [5]:
max_n = df['n'].max()
max_n

535

In [6]:
%%time
sex_series = df.Sex.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
sex_stacked = np.vstack(sex_series)

length_series = df.Length.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
length_stacked = np.vstack(length_series)

male_ind_series = df.Male_ind.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
male_ind_stacked = np.vstack(male_ind_series)

female_ind_series = df.Female_ind.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
female_ind_stacked = np.vstack(female_ind_series)

CPU times: user 6.65 ms, sys: 791 µs, total: 7.45 ms
Wall time: 12 ms


In [7]:
test_dataset = xr.Dataset(dict(
    Sex=(['haul', 'index'], sex_stacked), 
    Length=(['haul', 'index'], length_stacked),
    Male_ind=(['haul', 'index'], male_ind_stacked),
    Female_ind=(['haul', 'index'], female_ind_stacked), 
    n=(['haul'], df.n.to_numpy()),
    meanlen=(['haul'], df.meanlen.to_numpy()),
    stdlen=(['haul'], df.stdlen.to_numpy()),
    TS_lin=(['haul'], df.TS_lin.to_numpy()),
    TS_sd=(['haul'], df.TS_sd.to_numpy()),
    TS_log=(['haul'], df.TS_log.to_numpy()),
    nM=(['haul'], df.nM.to_numpy()),
    nF=(['haul'], df.nF.to_numpy())
), coords={
    'haul': df.index.to_numpy(),
    'index' : range(max_n)
})

In [8]:
test_dataset

<xarray.Dataset>
Dimensions:     (haul: 55, index: 535)
Coordinates:
  * haul        (haul) int64 1 3 7 8 9 10 11 12 13 ... 63 65 66 67 69 70 71 73
  * index       (index) int64 0 1 2 3 4 5 6 7 ... 528 529 530 531 532 533 534
Data variables:
    Sex         (haul, index) float64 2.0 2.0 2.0 2.0 2.0 ... nan nan nan nan
    Length      (haul, index) float64 20.0 20.0 20.0 20.0 ... nan nan nan nan
    Male_ind    (haul, index) float64 148.0 149.0 150.0 151.0 ... nan nan nan
    Female_ind  (haul, index) float64 0.0 1.0 2.0 3.0 4.0 ... nan nan nan nan
    n           (haul) int64 342 535 167 161 127 230 ... 137 289 293 187 238 342
    meanlen     (haul) float64 23.39 20.95 22.24 23.27 ... 43.29 44.53 48.77
    stdlen      (haul) float64 2.126 1.401 1.248 1.782 ... 3.778 2.557 4.106
    TS_lin      (haul) float64 -40.59 -41.56 -41.04 ... -35.24 -35.01 -34.21
    TS_sd       (haul) float64 0.7475 0.7043 0.489 ... 0.7785 0.5122 0.6952
    TS_log      (haul) float64 -40.65 -41.6 -41.07 ... -35.31 -35.04 -34.27
    nM          (haul) int64 194 97 95 93 64 125 119 ... 63 161 133 105 159 144
    nF          (haul) int64 148 128 72 68 63 101 118 ... 74 128 160 82 79 198

In [9]:
test_dataset.nbytes

949840

In [10]:
# %%time
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_catch_US'])
# df = epro_2019.process_catch_data(out)

In [11]:
# %%time
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_specimen_US'])
# df = epro_2019.process_specimen_data(out, 0)
# df.head()

In [12]:
# catch_us_df, length_us_df, trawl_us_df, gear_us_df, specimen_us_df = epro_2019.load_biological_data_us()
# catch_can_df, length_can_df, trawl_can_df, gear_can_df, specimen_can_df = epro_2019.load_biological_data_canada()